## Gesture Recognition
To build a 3D Conv model that will be able to predict the 5 gestures correctly.

__1. Import all necessary libraries__

In [31]:
import numpy as np
import os
from skimage.io import imread
from skimage.transform import resize
import datetime

In [32]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [33]:
import cv2
import matplotlib.pyplot as plt

__2. Load the training and testing data__

In [34]:
train_doc = np.random.permutation(open('/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/datasets/Project_data/val.csv').readlines())
batch_size = 16

In [35]:
lower = np.array([0, 48, 80], dtype = "uint8") ## lower limit for skin colour in HSV format
upper = np.array([20, 255, 255], dtype = "uint8")## lower limit for skin colour in HSV format

__3. Define generator function__

In [36]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [1,2,5,6,8,10,12,13,15,17,18,20,22,24,26,27,28,29]
    while True:
        t = np.random.permutation(folder_list)
        num_batches = int(len(t)/batch_size)
        for batch in range(num_batches):
            batch_data = np.zeros((batch_size,18,86,86,3))
            batch_labels = np.zeros((batch_size,5))
            for folder in range(batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = cv2.imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])# loading image
                    converted = cv2.cvtColor(image, cv2.COLOR_BGR2HSV) # convert image to HSV format
                    skinMask = cv2.inRange(converted, lower, upper) # setting boundaries to get the skin colour
                    image = cv2.bitwise_and(image, image, mask = skinMask) # retaining only the region with skin colour
                    
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(86,86)).astype(np.float32)
                    else:
                        image = resize(image,(86,86)).astype(np.float32)
                    # normalize the image to get the range of data in between 0 to 255
                    image = cv2.normalize(image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8) 
                    # bring each channel to mean
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 50 
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 100
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 140
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

        if (len(t)%batch_size) != 0:
            batch_data = np.zeros((len(t)%batch_size,18,86,86,3))
            batch_labels = np.zeros((len(t)%batch_size,5))
            for folder in range(len(t)%batch_size):
                imgs = os.listdir(source_path+'/'+ t[folder + (num_batches*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx):
                    image = cv2.imread(source_path+'/'+ t[folder + (num_batches*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    converted = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
                    skinMask = cv2.inRange(converted, lower, upper)
                    image = cv2.bitwise_and(image, image, mask = skinMask)
                    if image.shape[1] == 160:
                        image = resize(image[:,20:140,:],(86,86)).astype(np.float32)
                    else:
                        image = resize(image,(86,86)).astype(np.float32)
                    image = cv2.normalize(image, None, alpha = 0, beta = 255, norm_type = cv2.NORM_MINMAX, dtype = cv2.CV_32F).astype(np.uint8)
                    batch_data[folder,idx,:,:,0] = image[:,:,0] - 50
                    batch_data[folder,idx,:,:,1] = image[:,:,1] - 100
                    batch_data[folder,idx,:,:,2] = image[:,:,2] - 140

                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1

            yield batch_data, batch_labels

__4. Define epochs,train and validation folder path__ 

In [37]:
curr_dt_time = datetime.datetime.now()
train_path = '/datasets/Project_data/train'
val_path = '/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


__5. create model__

In [38]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers,regularizers

model = Sequential()
model.add(Conv3D(64, (3,3,3), strides=(1,1,1), padding='same', input_shape=(18,84,84,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,1), strides=(2,2,1)))

model.add(Conv3D(128, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

# model.add(Dropout(0.25))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling3D(pool_size=(2,2,2), strides=(2,2,2)))

model.add(Conv3D(256, (3,3,3), strides=(1,1,1), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

__5. compile model with optimizers__

In [39]:
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.7, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 18, 84, 84, 64)    5248      
_________________________________________________________________
batch_normalization_24 (Batc (None, 18, 84, 84, 64)    256       
_________________________________________________________________
activation_24 (Activation)   (None, 18, 84, 84, 64)    0         
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 9, 42, 84, 64)     0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 9, 42, 84, 128)    221312    
_________________________________________________________________
batch_normalization_25 (Batc (None, 9, 42, 84, 128)    512       
_________________________________________________________________
activation_25 (Activation)   (None, 9, 42, 84, 128)   

In [40]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

__6.Defining callbacks__

In [41]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1, mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
callbacks_list = [checkpoint, LR]

__7. Calculating number of steps per epoch in both train and validation__

In [42]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

__Fitting the model__

In [43]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /datasets/Project_data/train ; batch size = 16
Epoch 1/30
42/42 [==============================] - ETA: 0s - loss: 4.3029 - categorical_accuracy: 0.2896Source path =  /datasets/Project_data/val ; batch size = 16

Epoch 00001: saving model to model_init_2021-07-3010_16_16.986489/model-00001-4.30287-0.28959-4.09462-0.18000.h5
42/42 [==============================] - 94s 2s/step - loss: 4.3029 - categorical_accuracy: 0.2896 - val_loss: 4.0946 - val_categorical_accuracy: 0.1800
Epoch 2/30
42/42 [==============================] - ETA: 0s - loss: 3.7757 - categorical_accuracy: 0.5113
Epoch 00002: saving model to model_init_2021-07-3010_16_16.986489/model-00002-3.77571-0.51131-4.00088-0.38000.h5
42/42 [==============================] - 93s 2s/step - loss: 3.7757 - categorical_accuracy: 0.5113 - val_loss: 4.0009 - val_categorical_accuracy: 0.3800
Epoch 3/30
42/42 [==============================] - ETA: 0s - loss: 3.5426 - categorical_accuracy: 0.5867
Epoch 00003: saving model to